In [ ]:
# IMPORT LIBRARIES
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import statsmodels.api as sm
%matplotlib inline
import sklearn
import plotly.figure_factory as ff
import warnings
import os
import plotly.express as px
import xgboost as xgb
from datetime import timedelta
from matplotlib.colors import LinearSegmentedColormap
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, QuantileTransformer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import StandardScaler 
from sklearn.metrics import mean_squared_error
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from statsmodels.tsa.stattools import pacf
from sklearn.utils import resample
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.inspection import permutation_importance
from sklearn.datasets import make_friedman1
from sklearn.feature_selection import RFECV
from sklearn.model_selection import KFold
from sklearn.svm import LinearSVC
from sklearn.feature_selection import SelectFromModel
from sklearn.svm import SVR
from sklearn.linear_model import LogisticRegression
from statsmodels.tsa.api import ExponentialSmoothing, SimpleExpSmoothing, Holt
from sklearn.cluster import KMeans

In [ ]:
# Import Datasets
demographics = pd.read_csv('demographics.csv')
purchases = pd.read_csv('purchases.csv')
variable_mapping = pd.read_csv('variable_mapping.csv')

### MISSING VALUE IMPUTATION FOR PURCHASES DATA 

In [ ]:
# Check if these is any null values in the purchases dataset
print(purchases.isnull().sum(axis = 0))

In [ ]:
# Process Missing Value Imputation for Units feature - using 10 iteration
list1 = ['UNITS']
imp = IterativeImputer(max_iter=10, random_state=0)
imp.fit(purchases[list1])
IterativeImputer(random_state=0)
# Transform the dataset based on fitted units feature
purchases[list1] = imp.transform(purchases[list1])
# Check the operation
print(purchases.isnull().sum(axis = 0))

In [ ]:
#K Generating Year-Month-Day columns for further analysis
purchases['TRANSACTION_DT'] = pd.to_datetime(purchases['TRANSACTION_DT'])
purchases['Year'] = purchases['TRANSACTION_DT'].dt.year
purchases['Month'] = purchases['TRANSACTION_DT'].dt.month

# PREDICTIVE MODEL FOR CHURN USERS

### Prepare the Time Series Data

In [ ]:
#Combine the year and month column
purchases["Year_Month"]=purchases["Year"].astype(str)+"-"+purchases["Month"].astype(str)

#Set up the month order
month_order = ["2018-1","2018-2","2018-3","2018-4","2018-5","2018-6","2018-7","2018-8",
               "2018-9","2018-10","2018-11","2018-12","2019-1","2019-2","2019-3","2019-4",
               "2019-5","2019-6","2019-7","2019-8","2019-9","2019-10","2019-11","2019-12"]

In [ ]:
#Prepare the data for time series analysis
ts = purchases.groupby('USER_ID')['Year_Month'].value_counts().unstack().fillna(0)[month_order]
# Reset index to utilize 'USER_ID' feature
ts = ts.reset_index()
ts.head()

In [ ]:
# Generate 11 identical year blocks based on 11 different months (the aim is to distill as much 'churn' user data as possible for the model)
mo_1 = purchases[(purchases['TRANSACTION_DT'] > '2018-02-01') & (purchases['TRANSACTION_DT'] < '2019-02-01')]
mo_2 = purchases[(purchases['TRANSACTION_DT'] > '2018-03-01') & (purchases['TRANSACTION_DT'] < '2019-03-01')]
mo_3 = purchases[(purchases['TRANSACTION_DT'] > '2018-04-01') & (purchases['TRANSACTION_DT'] < '2019-04-01')]
mo_4 = purchases[(purchases['TRANSACTION_DT'] > '2018-05-01') & (purchases['TRANSACTION_DT'] < '2019-05-01')]
mo_5 = purchases[(purchases['TRANSACTION_DT'] > '2018-06-01') & (purchases['TRANSACTION_DT'] < '2019-06-01')]
mo_6 = purchases[(purchases['TRANSACTION_DT'] > '2018-07-01') & (purchases['TRANSACTION_DT'] < '2019-07-01')]
mo_7 = purchases[(purchases['TRANSACTION_DT'] > '2018-08-01') & (purchases['TRANSACTION_DT'] < '2019-08-01')]
mo_8 = purchases[(purchases['TRANSACTION_DT'] > '2018-09-01') & (purchases['TRANSACTION_DT'] < '2019-09-01')]
mo_9 = purchases[(purchases['TRANSACTION_DT'] > '2018-10-01') & (purchases['TRANSACTION_DT'] < '2019-10-01')]
mo_10 = purchases[(purchases['TRANSACTION_DT'] > '2018-11-01') & (purchases['TRANSACTION_DT'] < '2019-11-01')]
mo_11 = purchases[(purchases['TRANSACTION_DT'] > '2018-12-01') & (purchases['TRANSACTION_DT'] < '2019-12-01')]
len(mo_1.groupby('TRANSACTION_DT')['TRANSACTION_DT']) # just to check a random block

### Generate Recency and Frequency Features

In [ ]:
# Generate Recency and Frequency features for the 1st block
snapshot_date = mo_1['TRANSACTION_DT'].max() + timedelta(days=1)
mo_1 = mo_1.groupby(['USER_ID']).agg({'TRANSACTION_DT': lambda x: (snapshot_date - x.max()).days,'BASKET_ID': 'count'})
mo_1.rename(columns={'TRANSACTION_DT': 'Recency','BASKET_ID': 'Frequency'}, inplace=True)
# enerate Recency and Frequency features for the 2nd block
snapshot_date = mo_2['TRANSACTION_DT'].max() + timedelta(days=1)
mo_2 = mo_2.groupby(['USER_ID']).agg({'TRANSACTION_DT': lambda x: (snapshot_date - x.max()).days,'BASKET_ID': 'count'})
mo_2.rename(columns={'TRANSACTION_DT': 'Recency','BASKET_ID': 'Frequency'}, inplace=True)
# enerate Recency and Frequency features for the 3rd block
snapshot_date = mo_3['TRANSACTION_DT'].max() + timedelta(days=1)
mo_3 = mo_3.groupby(['USER_ID']).agg({'TRANSACTION_DT': lambda x: (snapshot_date - x.max()).days,'BASKET_ID': 'count'})
mo_3.rename(columns={'TRANSACTION_DT': 'Recency','BASKET_ID': 'Frequency'}, inplace=True)
# enerate Recency and Frequency features for the 4th block
snapshot_date = mo_4['TRANSACTION_DT'].max() + timedelta(days=1)
mo_4 = mo_4.groupby(['USER_ID']).agg({'TRANSACTION_DT': lambda x: (snapshot_date - x.max()).days,'BASKET_ID': 'count'})
mo_4.rename(columns={'TRANSACTION_DT': 'Recency','BASKET_ID': 'Frequency'}, inplace=True)
# enerate Recency and Frequency features for the 5th block
snapshot_date = mo_5['TRANSACTION_DT'].max() + timedelta(days=1)
mo_5 = mo_5.groupby(['USER_ID']).agg({'TRANSACTION_DT': lambda x: (snapshot_date - x.max()).days,'BASKET_ID': 'count'})
mo_5.rename(columns={'TRANSACTION_DT': 'Recency','BASKET_ID': 'Frequency'}, inplace=True)
# enerate Recency and Frequency features for the 6th block
snapshot_date = mo_6['TRANSACTION_DT'].max() + timedelta(days=1)
mo_6 = mo_6.groupby(['USER_ID']).agg({'TRANSACTION_DT': lambda x: (snapshot_date - x.max()).days,'BASKET_ID': 'count'})
mo_6.rename(columns={'TRANSACTION_DT': 'Recency','BASKET_ID': 'Frequency'}, inplace=True)
# enerate Recency and Frequency features for the 7th block
snapshot_date = mo_7['TRANSACTION_DT'].max() + timedelta(days=1)
mo_7 = mo_7.groupby(['USER_ID']).agg({'TRANSACTION_DT': lambda x: (snapshot_date - x.max()).days,'BASKET_ID': 'count'})
mo_7.rename(columns={'TRANSACTION_DT': 'Recency','BASKET_ID': 'Frequency'}, inplace=True)
# enerate Recency and Frequency features for the 8th block
snapshot_date = mo_8['TRANSACTION_DT'].max() + timedelta(days=1)
mo_8 = mo_8.groupby(['USER_ID']).agg({'TRANSACTION_DT': lambda x: (snapshot_date - x.max()).days,'BASKET_ID': 'count'})
mo_8.rename(columns={'TRANSACTION_DT': 'Recency','BASKET_ID': 'Frequency'}, inplace=True)
# enerate Recency and Frequency features for the 9th block
snapshot_date = mo_9['TRANSACTION_DT'].max() + timedelta(days=1)
mo_9 = mo_9.groupby(['USER_ID']).agg({'TRANSACTION_DT': lambda x: (snapshot_date - x.max()).days,'BASKET_ID': 'count'})
mo_9.rename(columns={'TRANSACTION_DT': 'Recency','BASKET_ID': 'Frequency'}, inplace=True)
# enerate Recency and Frequency features for the 10th block
snapshot_date = mo_10['TRANSACTION_DT'].max() + timedelta(days=1)
mo_10 = mo_10.groupby(['USER_ID']).agg({'TRANSACTION_DT': lambda x: (snapshot_date - x.max()).days,'BASKET_ID': 'count'})
mo_10.rename(columns={'TRANSACTION_DT': 'Recency','BASKET_ID': 'Frequency'}, inplace=True)
# enerate Recency and Frequency features for the 11th block
snapshot_date = mo_11['TRANSACTION_DT'].max() + timedelta(days=1)
mo_11 = mo_11.groupby(['USER_ID']).agg({'TRANSACTION_DT': lambda x: (snapshot_date - x.max()).days,'BASKET_ID': 'count'})
mo_11.rename(columns={'TRANSACTION_DT': 'Recency','BASKET_ID': 'Frequency'}, inplace=True)

In [ ]:
mo_11.head(3) # just to check

### Time Series Data Preparation for each Month - making them identical

In [ ]:
# Read prevıously generated ts_data as ts
ts = pd.read_csv('ts_data.csv')
# Create a derivative dataset solely composed of months
ts2 = ts.iloc[:,1:]
ts2[ts2['2019-5'] == 0].head(3)

In [ ]:
# 1st yearly block starting with 2018-2 
month1 = ts2.iloc[:,np.r_[0,2:15]]
# Integrate User_ID features Recency and Frequency
month1 = month1.merge(mo_1,left_on='USER_ID', right_on='USER_ID') 
month1.head(2)

In [ ]:
# 2nd yearly block starting with 2018-3 
month2 = ts2.iloc[:,np.r_[0,3:16]]
# Integrate User_ID features Recency and Frequency
month2 = month2.merge(mo_2,left_on='USER_ID', right_on='USER_ID') 
month2.head(2)

In [ ]:
# 3rd yearly block starting with 2018-4 
month3 = ts2.iloc[:,np.r_[0,4:17]]
# Integrate User_ID features Recency and Frequency
month3 = month3.merge(mo_3,left_on='USER_ID', right_on='USER_ID') 
month3.head(2)

In [ ]:
# 4th yearly block starting with 2018-5 
month4 = ts2.iloc[:,np.r_[0,5:18]]
# Integrate User_ID features Recency and Frequency
month4 = month4.merge(mo_4,left_on='USER_ID', right_on='USER_ID') 
month4.head(2)

In [ ]:
# 5th yearly block starting with 2018-6 
month5 = ts2.iloc[:,np.r_[0,6:19]]
# Integrate User_ID features Recency and Frequency
month5 = month5.merge(mo_5,left_on='USER_ID', right_on='USER_ID') 
month5.head(2)

In [ ]:
# 6th yearly block starting with 2018-7 
month6 = ts2.iloc[:,np.r_[0,7:20]]
# Integrate User_ID features Recency and Frequency
month6 = month6.merge(mo_6,left_on='USER_ID', right_on='USER_ID') 
month6.head(2)

In [ ]:
# 7th yearly block starting with 2018-8 
month7 = ts2.iloc[:,np.r_[0,8:21]]
# Integrate User_ID features Recency and Frequency
month7 = month7.merge(mo_7,left_on='USER_ID', right_on='USER_ID') 
month7.head(2)

In [ ]:
# 8th yearly block starting with 2018-9 
month8 = ts2.iloc[:,np.r_[0,9:22]]
# Integrate User_ID features Recency and Frequency
month8 = month8.merge(mo_8,left_on='USER_ID', right_on='USER_ID') 
month8.head(2)

In [ ]:
# 9th yearly block starting with 2018-10 
month9 = ts2.iloc[:,np.r_[0,10:23]]
# Integrate User_ID features Recency and Frequency
month9 = month9.merge(mo_9,left_on='USER_ID', right_on='USER_ID') 
month9.head(2)

In [ ]:
# 10th yearly block starting with 2018-11 
month10 = ts2.iloc[:,np.r_[0,11:24]]
# Integrate User_ID features Recency and Frequency
month10 = month10.merge(mo_10,left_on='USER_ID', right_on='USER_ID') 
month10.head(2)

In [ ]:
# 11th yearly block starting with 2018-12 
month11 = ts2.iloc[:,np.r_[0,12:25]]
# Integrate User_ID features Recency and Frequency
month11 = month11.merge(mo_11,left_on='USER_ID', right_on='USER_ID') 
month11.head(2)

In [ ]:
# Delete previously created dummy datasets
del mo_1;del mo_2; del mo_3; del mo_4; del mo_5; del mo_6; del mo_7; del mo_8; del mo_9; del mo_10; del mo_11  

### Define Active-Churn-New Users

In [ ]:
# Create a for loop to define User Status for the year block  - month1 
User_Status = []
month = month1
rows = len(month)
# Define formulas based on Active, Churn customer definitions of Company X
for x in range(0,rows):
    if  (month.iloc[x,1] > 0)&(month.iloc[x,2] > 0)&(month.iloc[x,3] > 0)&(month.iloc[x,4] > 0)&(month.iloc[x,5] > 0)&(month.iloc[x,6] > 0)&(month.iloc[x,7] > 0)&(month.iloc[x,8] > 0)&(month.iloc[x,9] > 0)&(month.iloc[x,10] > 0)&(month.iloc[x,11] > 0)&(month.iloc[x,12] > 0)&(month.iloc[x,13] > 0):
        val = 1 # ACTIVE USER    
    elif (month.iloc[x,1] > 0)&(month.iloc[x,2] > 0)&(month.iloc[x,3] > 0)&(month.iloc[x,4] > 0)&(month.iloc[x,5] > 0)&(month.iloc[x,6] > 0)&(month.iloc[x,7] > 0)&(month.iloc[x,8] > 0)&(month.iloc[x,9] > 0)&(month.iloc[x,10] > 0)&(month.iloc[x,11] > 0)&(month.iloc[x,12] > 0)&(month.iloc[x,13] == 0):
        val = 0  # JUST CHURNED   
    elif (month.iloc[x,1] > 0)&(month.iloc[x,2] > 0)&(month.iloc[x,3] > 0)&(month.iloc[x,4] > 0)&(month.iloc[x,5] > 0)&(month.iloc[x,6] > 0)&(month.iloc[x,7] > 0)&(month.iloc[x,8] > 0)&(month.iloc[x,9] > 0)&(month.iloc[x,10] > 0)&(month.iloc[x,11] > 0)&(month.iloc[x,12] == 0):
        val = 2  # ALREADY CHURNED (DROP FROM THE MODEL BEFOREHAND)
    elif (month.iloc[x,12] == 0)&(month.iloc[x,11] == 0)&(month.iloc[x,10] == 0):
        val = 2  # ALREADY CHURNED  (DROP FROM THE MODEL BEFOREHAND)
    elif (month.iloc[x,12] > 0)&(month.iloc[x,11] == 0)&(month.iloc[x,10] == 0)&(month.iloc[x,9] == 0):
        val = 3  # NEW CUSTOMER (DON'T INVOLVE INTO MODEL -DROP- )
    else:
        val = 1 # REST COULD BE CONSIDERED ACTIVE
    User_Status.append(val)
User_Status = pd.DataFrame(User_Status,columns = {"User_Status"})
# Integrate user definitions with pre generated time series dataset 
ts3_1 = pd.concat([month1,User_Status],axis=1)
print(ts3_1.groupby('User_Status')['2019-2'].count())
################################################################################################################
# Create a for loop to define User Status for the year block  - month2
User_Status = []
month = month2
rows = len(month)
# Define formulas based on Active, Churn customer definitions of Company X
for x in range(0,rows):
    if  (month.iloc[x,1] > 0)&(month.iloc[x,2] > 0)&(month.iloc[x,3] > 0)&(month.iloc[x,4] > 0)&(month.iloc[x,5] > 0)&(month.iloc[x,6] > 0)&(month.iloc[x,7] > 0)&(month.iloc[x,8] > 0)&(month.iloc[x,9] > 0)&(month.iloc[x,10] > 0)&(month.iloc[x,11] > 0)&(month.iloc[x,12] > 0)&(month.iloc[x,13] > 0):
        val = 1 # ACTIVE USER    
    elif (month.iloc[x,1] > 0)&(month.iloc[x,2] > 0)&(month.iloc[x,3] > 0)&(month.iloc[x,4] > 0)&(month.iloc[x,5] > 0)&(month.iloc[x,6] > 0)&(month.iloc[x,7] > 0)&(month.iloc[x,8] > 0)&(month.iloc[x,9] > 0)&(month.iloc[x,10] > 0)&(month.iloc[x,11] > 0)&(month.iloc[x,12] > 0)&(month.iloc[x,13] == 0):
        val = 0  # JUST CHURNED   
    elif (month.iloc[x,1] > 0)&(month.iloc[x,2] > 0)&(month.iloc[x,3] > 0)&(month.iloc[x,4] > 0)&(month.iloc[x,5] > 0)&(month.iloc[x,6] > 0)&(month.iloc[x,7] > 0)&(month.iloc[x,8] > 0)&(month.iloc[x,9] > 0)&(month.iloc[x,10] > 0)&(month.iloc[x,11] > 0)&(month.iloc[x,12] == 0):
        val = 2  # ALREADY CHURNED 
    elif (month.iloc[x,12] == 0)&(month.iloc[x,11] == 0)&(month.iloc[x,10] == 0):
        val = 2  # ALREADY CHURNED
    elif (month.iloc[x,12] > 0)&(month.iloc[x,11] == 0)&(month.iloc[x,10] == 0)&(month.iloc[x,9] == 0):
        val = 3  # NEW CUSTOMER (DON'T INVOLVE INTO MODEL -DROP- )
    else:
        val = 1 # REST COULD BE CONSIDERED ACTIVE
    User_Status.append(val)
User_Status = pd.DataFrame(User_Status,columns = {"User_Status"})
# Integrate user definitions with pre generated time series dataset 
ts3_2 = pd.concat([month2,User_Status],axis=1)
print(ts3_2.groupby('User_Status')['2019-3'].count())
#################################################################################################################
# Create a for loop to define User Status for the year block  - month3 
User_Status = []
month = month3
rows = len(month)
# Define formulas based on Active, Churn customer definitions of Company X
for x in range(0,rows):
    if  (month.iloc[x,1] > 0)&(month.iloc[x,2] > 0)&(month.iloc[x,3] > 0)&(month.iloc[x,4] > 0)&(month.iloc[x,5] > 0)&(month.iloc[x,6] > 0)&(month.iloc[x,7] > 0)&(month.iloc[x,8] > 0)&(month.iloc[x,9] > 0)&(month.iloc[x,10] > 0)&(month.iloc[x,11] > 0)&(month.iloc[x,12] > 0)&(month.iloc[x,13] > 0):
        val = 1 # ACTIVE USER    
    elif (month.iloc[x,1] > 0)&(month.iloc[x,2] > 0)&(month.iloc[x,3] > 0)&(month.iloc[x,4] > 0)&(month.iloc[x,5] > 0)&(month.iloc[x,6] > 0)&(month.iloc[x,7] > 0)&(month.iloc[x,8] > 0)&(month.iloc[x,9] > 0)&(month.iloc[x,10] > 0)&(month.iloc[x,11] > 0)&(month.iloc[x,12] > 0)&(month.iloc[x,13] == 0):
        val = 0  # JUST CHURNED   
    elif (month.iloc[x,1] > 0)&(month.iloc[x,2] > 0)&(month.iloc[x,3] > 0)&(month.iloc[x,4] > 0)&(month.iloc[x,5] > 0)&(month.iloc[x,6] > 0)&(month.iloc[x,7] > 0)&(month.iloc[x,8] > 0)&(month.iloc[x,9] > 0)&(month.iloc[x,10] > 0)&(month.iloc[x,11] > 0)&(month.iloc[x,12] == 0):
        val = 2  # ALREADY CHURNED 
    elif (month.iloc[x,12] == 0)&(month.iloc[x,11] == 0)&(month.iloc[x,10] == 0):
        val = 2  # ALREADY CHURNED
    elif (month.iloc[x,12] > 0)&(month.iloc[x,11] == 0)&(month.iloc[x,10] == 0)&(month.iloc[x,9] == 0):
        val = 3  # NEW CUSTOMER (DON'T INVOLVE INTO MODEL -DROP- )
    else:
        val = 1 # REST COULD BE CONSIDERED ACTIVE
    User_Status.append(val)
User_Status = pd.DataFrame(User_Status,columns = {"User_Status"})
# Integrate user definitions with pre generated time series dataset 
ts3_3 = pd.concat([month3,User_Status],axis=1)
print(ts3_3.groupby('User_Status')['2019-4'].count())
#################################################################################################################
# Create a for loop to define User Status for the year block  - month4 
User_Status = []
month = month4
rows = len(month)
# Define formulas based on Active, Churn customer definitions of Company X
for x in range(0,rows):
    if  (month.iloc[x,1] > 0)&(month.iloc[x,2] > 0)&(month.iloc[x,3] > 0)&(month.iloc[x,4] > 0)&(month.iloc[x,5] > 0)&(month.iloc[x,6] > 0)&(month.iloc[x,7] > 0)&(month.iloc[x,8] > 0)&(month.iloc[x,9] > 0)&(month.iloc[x,10] > 0)&(month.iloc[x,11] > 0)&(month.iloc[x,12] > 0)&(month.iloc[x,13] > 0):
        val = 1 # ACTIVE USER    
    elif (month.iloc[x,1] > 0)&(month.iloc[x,2] > 0)&(month.iloc[x,3] > 0)&(month.iloc[x,4] > 0)&(month.iloc[x,5] > 0)&(month.iloc[x,6] > 0)&(month.iloc[x,7] > 0)&(month.iloc[x,8] > 0)&(month.iloc[x,9] > 0)&(month.iloc[x,10] > 0)&(month.iloc[x,11] > 0)&(month.iloc[x,12] > 0)&(month.iloc[x,13] == 0):
        val = 0  # JUST CHURNED   
    elif (month.iloc[x,1] > 0)&(month.iloc[x,2] > 0)&(month.iloc[x,3] > 0)&(month.iloc[x,4] > 0)&(month.iloc[x,5] > 0)&(month.iloc[x,6] > 0)&(month.iloc[x,7] > 0)&(month.iloc[x,8] > 0)&(month.iloc[x,9] > 0)&(month.iloc[x,10] > 0)&(month.iloc[x,11] > 0)&(month.iloc[x,12] == 0):
        val = 2  # ALREADY CHURNED 
    elif (month.iloc[x,12] == 0)&(month.iloc[x,11] == 0)&(month.iloc[x,10] == 0):
        val = 2  # ALREADY CHURNED
    elif (month.iloc[x,12] > 0)&(month.iloc[x,11] == 0)&(month.iloc[x,10] == 0)&(month.iloc[x,9] == 0):
        val = 3  # NEW CUSTOMER (DON'T INVOLVE INTO MODEL -DROP- )
    else:
        val = 1 # REST COULD BE CONSIDERED ACTIVE
    User_Status.append(val)
User_Status = pd.DataFrame(User_Status,columns = {"User_Status"})
# Integrate user definitions with pre generated time series dataset 
ts3_4 = pd.concat([month4,User_Status],axis=1)
print(ts3_4.groupby('User_Status')['2019-5'].count())
#################################################################################################################
# Create a for loop to define User Status for the year block  - month5 
User_Status = []
month = month5
rows = len(month)
# Define formulas based on Active, Churn customer definitions of Company X
for x in range(0,rows):
    if  (month.iloc[x,1] > 0)&(month.iloc[x,2] > 0)&(month.iloc[x,3] > 0)&(month.iloc[x,4] > 0)&(month.iloc[x,5] > 0)&(month.iloc[x,6] > 0)&(month.iloc[x,7] > 0)&(month.iloc[x,8] > 0)&(month.iloc[x,9] > 0)&(month.iloc[x,10] > 0)&(month.iloc[x,11] > 0)&(month.iloc[x,12] > 0)&(month.iloc[x,13] > 0):
        val = 1 # ACTIVE USER    
    elif (month.iloc[x,1] > 0)&(month.iloc[x,2] > 0)&(month.iloc[x,3] > 0)&(month.iloc[x,4] > 0)&(month.iloc[x,5] > 0)&(month.iloc[x,6] > 0)&(month.iloc[x,7] > 0)&(month.iloc[x,8] > 0)&(month.iloc[x,9] > 0)&(month.iloc[x,10] > 0)&(month.iloc[x,11] > 0)&(month.iloc[x,12] > 0)&(month.iloc[x,13] == 0):
        val = 0  # JUST CHURNED   
    elif (month.iloc[x,1] > 0)&(month.iloc[x,2] > 0)&(month.iloc[x,3] > 0)&(month.iloc[x,4] > 0)&(month.iloc[x,5] > 0)&(month.iloc[x,6] > 0)&(month.iloc[x,7] > 0)&(month.iloc[x,8] > 0)&(month.iloc[x,9] > 0)&(month.iloc[x,10] > 0)&(month.iloc[x,11] > 0)&(month.iloc[x,12] == 0):
        val = 2  # ALREADY CHURNED 
    elif (month.iloc[x,12] == 0)&(month.iloc[x,11] == 0)&(month.iloc[x,10] == 0):
        val = 2  # ALREADY CHURNED
    elif (month.iloc[x,12] > 0)&(month.iloc[x,11] == 0)&(month.iloc[x,10] == 0)&(month.iloc[x,9] == 0):
        val = 3  # NEW CUSTOMER (DON'T INVOLVE INTO MODEL -DROP- )
    else:
        val = 1 # REST COULD BE CONSIDERED ACTIVE
    User_Status.append(val)
User_Status = pd.DataFrame(User_Status,columns = {"User_Status"})
# Integrate user definitions with pre generated time series dataset 
ts3_5 = pd.concat([month5,User_Status],axis=1)
print(ts3_5.groupby('User_Status')['2019-6'].count())
#################################################################################################################
# Create a for loop to define User Status for the year block  - month6 
User_Status = []
month = month6
rows = len(month)
# Define formulas based on Active, Churn customer definitions of Company X
for x in range(0,rows):
    if  (month.iloc[x,1] > 0)&(month.iloc[x,2] > 0)&(month.iloc[x,3] > 0)&(month.iloc[x,4] > 0)&(month.iloc[x,5] > 0)&(month.iloc[x,6] > 0)&(month.iloc[x,7] > 0)&(month.iloc[x,8] > 0)&(month.iloc[x,9] > 0)&(month.iloc[x,10] > 0)&(month.iloc[x,11] > 0)&(month.iloc[x,12] > 0)&(month.iloc[x,13] > 0):
        val = 1 # ACTIVE USER    
    elif (month.iloc[x,1] > 0)&(month.iloc[x,2] > 0)&(month.iloc[x,3] > 0)&(month.iloc[x,4] > 0)&(month.iloc[x,5] > 0)&(month.iloc[x,6] > 0)&(month.iloc[x,7] > 0)&(month.iloc[x,8] > 0)&(month.iloc[x,9] > 0)&(month.iloc[x,10] > 0)&(month.iloc[x,11] > 0)&(month.iloc[x,12] > 0)&(month.iloc[x,13] == 0):
        val = 0  # JUST CHURNED   
    elif (month.iloc[x,1] > 0)&(month.iloc[x,2] > 0)&(month.iloc[x,3] > 0)&(month.iloc[x,4] > 0)&(month.iloc[x,5] > 0)&(month.iloc[x,6] > 0)&(month.iloc[x,7] > 0)&(month.iloc[x,8] > 0)&(month.iloc[x,9] > 0)&(month.iloc[x,10] > 0)&(month.iloc[x,11] > 0)&(month.iloc[x,12] == 0):
        val = 2  # ALREADY CHURNED 
    elif (month.iloc[x,12] == 0)&(month.iloc[x,11] == 0)&(month.iloc[x,10] == 0):
        val = 2  # ALREADY CHURNED
    elif (month.iloc[x,12] > 0)&(month.iloc[x,11] == 0)&(month.iloc[x,10] == 0)&(month.iloc[x,9] == 0):
        val = 3  # NEW CUSTOMER (DON'T INVOLVE INTO MODEL -DROP- )
    else:
        val = 1 # REST COULD BE CONSIDERED ACTIVE
    User_Status.append(val)
User_Status = pd.DataFrame(User_Status,columns = {"User_Status"})
# Integrate user definitions with pre generated time series dataset 
ts3_6 = pd.concat([month6,User_Status],axis=1)
print(ts3_6.groupby('User_Status')['2019-7'].count())
#################################################################################################################
# Create a for loop to define User Status for the year block  - month7 
User_Status = []
month = month7
rows = len(month)
# Define formulas based on Active, Churn customer definitions of Company X
for x in range(0,rows):
    if  (month.iloc[x,1] > 0)&(month.iloc[x,2] > 0)&(month.iloc[x,3] > 0)&(month.iloc[x,4] > 0)&(month.iloc[x,5] > 0)&(month.iloc[x,6] > 0)&(month.iloc[x,7] > 0)&(month.iloc[x,8] > 0)&(month.iloc[x,9] > 0)&(month.iloc[x,10] > 0)&(month.iloc[x,11] > 0)&(month.iloc[x,12] > 0)&(month.iloc[x,13] > 0):
        val = 1 # ACTIVE USER    
    elif (month.iloc[x,1] > 0)&(month.iloc[x,2] > 0)&(month.iloc[x,3] > 0)&(month.iloc[x,4] > 0)&(month.iloc[x,5] > 0)&(month.iloc[x,6] > 0)&(month.iloc[x,7] > 0)&(month.iloc[x,8] > 0)&(month.iloc[x,9] > 0)&(month.iloc[x,10] > 0)&(month.iloc[x,11] > 0)&(month.iloc[x,12] > 0)&(month.iloc[x,13] == 0):
        val = 0  # JUST CHURNED   
    elif (month.iloc[x,1] > 0)&(month.iloc[x,2] > 0)&(month.iloc[x,3] > 0)&(month.iloc[x,4] > 0)&(month.iloc[x,5] > 0)&(month.iloc[x,6] > 0)&(month.iloc[x,7] > 0)&(month.iloc[x,8] > 0)&(month.iloc[x,9] > 0)&(month.iloc[x,10] > 0)&(month.iloc[x,11] > 0)&(month.iloc[x,12] == 0):
        val = 2  # ALREADY CHURNED 
    elif (month.iloc[x,12] == 0)&(month.iloc[x,11] == 0)&(month.iloc[x,10] == 0):
        val = 2  # ALREADY CHURNED
    elif (month.iloc[x,12] > 0)&(month.iloc[x,11] == 0)&(month.iloc[x,10] == 0)&(month.iloc[x,9] == 0):
        val = 3  # NEW CUSTOMER (DON'T INVOLVE INTO MODEL -DROP- )
    else:
        val = 1 # REST COULD BE CONSIDERED ACTIVE
    User_Status.append(val)
User_Status = pd.DataFrame(User_Status,columns = {"User_Status"})
# Integrate user definitions with pre generated time series dataset 
ts3_7 = pd.concat([month7,User_Status],axis=1)
print(ts3_7.groupby('User_Status')['2019-8'].count())
#################################################################################################################
# Create a for loop to define User Status for the year block  - month8 
User_Status = []
month = month8
rows = len(month)
# Define formulas based on Active, Churn customer definitions of Company X
for x in range(0,rows):
    if  (month.iloc[x,1] > 0)&(month.iloc[x,2] > 0)&(month.iloc[x,3] > 0)&(month.iloc[x,4] > 0)&(month.iloc[x,5] > 0)&(month.iloc[x,6] > 0)&(month.iloc[x,7] > 0)&(month.iloc[x,8] > 0)&(month.iloc[x,9] > 0)&(month.iloc[x,10] > 0)&(month.iloc[x,11] > 0)&(month.iloc[x,12] > 0)&(month.iloc[x,13] > 0):
        val = 1 # ACTIVE USER    
    elif (month.iloc[x,1] > 0)&(month.iloc[x,2] > 0)&(month.iloc[x,3] > 0)&(month.iloc[x,4] > 0)&(month.iloc[x,5] > 0)&(month.iloc[x,6] > 0)&(month.iloc[x,7] > 0)&(month.iloc[x,8] > 0)&(month.iloc[x,9] > 0)&(month.iloc[x,10] > 0)&(month.iloc[x,11] > 0)&(month.iloc[x,12] > 0)&(month.iloc[x,13] == 0):
        val = 0  # JUST CHURNED   
    elif (month.iloc[x,1] > 0)&(month.iloc[x,2] > 0)&(month.iloc[x,3] > 0)&(month.iloc[x,4] > 0)&(month.iloc[x,5] > 0)&(month.iloc[x,6] > 0)&(month.iloc[x,7] > 0)&(month.iloc[x,8] > 0)&(month.iloc[x,9] > 0)&(month.iloc[x,10] > 0)&(month.iloc[x,11] > 0)&(month.iloc[x,12] == 0):
        val = 2  # ALREADY CHURNED 
    elif (month.iloc[x,12] == 0)&(month.iloc[x,11] == 0)&(month.iloc[x,10] == 0):
        val = 2  # ALREADY CHURNED
    elif (month.iloc[x,12] > 0)&(month.iloc[x,11] == 0)&(month.iloc[x,10] == 0)&(month.iloc[x,9] == 0):
        val = 3  # NEW CUSTOMER (DON'T INVOLVE INTO MODEL -DROP- )
    else:
        val = 1 # REST COULD BE CONSIDERED ACTIVE
    User_Status.append(val)
User_Status = pd.DataFrame(User_Status,columns = {"User_Status"})
# Integrate user definitions with pre generated time series dataset 
ts3_8 = pd.concat([month8,User_Status],axis=1)
print(ts3_8.groupby('User_Status')['2019-9'].count())
#################################################################################################################
# Create a for loop to define User Status for the year block  - month9 
User_Status = []
month = month9
rows = len(month)
# Define formulas based on Active, Churn customer definitions of Company X
for x in range(0,rows):
    if  (month.iloc[x,1] > 0)&(month.iloc[x,2] > 0)&(month.iloc[x,3] > 0)&(month.iloc[x,4] > 0)&(month.iloc[x,5] > 0)&(month.iloc[x,6] > 0)&(month.iloc[x,7] > 0)&(month.iloc[x,8] > 0)&(month.iloc[x,9] > 0)&(month.iloc[x,10] > 0)&(month.iloc[x,11] > 0)&(month.iloc[x,12] > 0)&(month.iloc[x,13] > 0):
        val = 1 # ACTIVE USER    
    elif (month.iloc[x,1] > 0)&(month.iloc[x,2] > 0)&(month.iloc[x,3] > 0)&(month.iloc[x,4] > 0)&(month.iloc[x,5] > 0)&(month.iloc[x,6] > 0)&(month.iloc[x,7] > 0)&(month.iloc[x,8] > 0)&(month.iloc[x,9] > 0)&(month.iloc[x,10] > 0)&(month.iloc[x,11] > 0)&(month.iloc[x,12] > 0)&(month.iloc[x,13] == 0):
        val = 0  # JUST CHURNED   
    elif (month.iloc[x,1] > 0)&(month.iloc[x,2] > 0)&(month.iloc[x,3] > 0)&(month.iloc[x,4] > 0)&(month.iloc[x,5] > 0)&(month.iloc[x,6] > 0)&(month.iloc[x,7] > 0)&(month.iloc[x,8] > 0)&(month.iloc[x,9] > 0)&(month.iloc[x,10] > 0)&(month.iloc[x,11] > 0)&(month.iloc[x,12] == 0):
        val = 2  # ALREADY CHURNED 
    elif (month.iloc[x,12] == 0)&(month.iloc[x,11] == 0)&(month.iloc[x,10] == 0):
        val = 2  # ALREADY CHURNED
    elif (month.iloc[x,12] > 0)&(month.iloc[x,11] == 0)&(month.iloc[x,10] == 0)&(month.iloc[x,9] == 0):
        val = 3  # NEW CUSTOMER (DON'T INVOLVE INTO MODEL -DROP- )
    else:
        val = 1 # REST COULD BE CONSIDERED ACTIVE
    User_Status.append(val)
User_Status = pd.DataFrame(User_Status,columns = {"User_Status"})
# Integrate user definitions with pre generated time series dataset 
ts3_9 = pd.concat([month9,User_Status],axis=1)
print(ts3_9.groupby('User_Status')['2019-10'].count())
#################################################################################################################
# Create a for loop to define User Status for the year block  - month10 
User_Status = []
month = month10
rows = len(month)
# Define formulas based on Active, Churn customer definitions of Company X
for x in range(0,rows):
    if  (month.iloc[x,1] > 0)&(month.iloc[x,2] > 0)&(month.iloc[x,3] > 0)&(month.iloc[x,4] > 0)&(month.iloc[x,5] > 0)&(month.iloc[x,6] > 0)&(month.iloc[x,7] > 0)&(month.iloc[x,8] > 0)&(month.iloc[x,9] > 0)&(month.iloc[x,10] > 0)&(month.iloc[x,11] > 0)&(month.iloc[x,12] > 0)&(month.iloc[x,13] > 0):
        val = 1 # ACTIVE USER    
    elif (month.iloc[x,1] > 0)&(month.iloc[x,2] > 0)&(month.iloc[x,3] > 0)&(month.iloc[x,4] > 0)&(month.iloc[x,5] > 0)&(month.iloc[x,6] > 0)&(month.iloc[x,7] > 0)&(month.iloc[x,8] > 0)&(month.iloc[x,9] > 0)&(month.iloc[x,10] > 0)&(month.iloc[x,11] > 0)&(month.iloc[x,12] > 0)&(month.iloc[x,13] == 0):
        val = 0  # JUST CHURNED   
    elif (month.iloc[x,1] > 0)&(month.iloc[x,2] > 0)&(month.iloc[x,3] > 0)&(month.iloc[x,4] > 0)&(month.iloc[x,5] > 0)&(month.iloc[x,6] > 0)&(month.iloc[x,7] > 0)&(month.iloc[x,8] > 0)&(month.iloc[x,9] > 0)&(month.iloc[x,10] > 0)&(month.iloc[x,11] > 0)&(month.iloc[x,12] == 0):
        val = 2  # ALREADY CHURNED 
    elif (month.iloc[x,12] == 0)&(month.iloc[x,11] == 0)&(month.iloc[x,10] == 0):
        val = 2  # ALREADY CHURNED
    elif (month.iloc[x,12] > 0)&(month.iloc[x,11] == 0)&(month.iloc[x,10] == 0)&(month.iloc[x,9] == 0):
        val = 3  # NEW CUSTOMER (DON'T INVOLVE INTO MODEL -DROP- )
    else:
        val = 1 # REST COULD BE CONSIDERED ACTIVE
    User_Status.append(val)
User_Status = pd.DataFrame(User_Status,columns = {"User_Status"})
# Integrate user definitions with pre generated time series dataset 
ts3_10 = pd.concat([month10,User_Status],axis=1)
print(ts3_10.groupby('User_Status')['2019-11'].count())
#################################################################################################################
# Create a for loop to define User Status for the year block  - month11 
User_Status = []
month = month11
rows = len(month)
# Define formulas based on Active, Churn customer definitions of Company X
for x in range(0,rows):
    if  (month.iloc[x,1] > 0)&(month.iloc[x,2] > 0)&(month.iloc[x,3] > 0)&(month.iloc[x,4] > 0)&(month.iloc[x,5] > 0)&(month.iloc[x,6] > 0)&(month.iloc[x,7] > 0)&(month.iloc[x,8] > 0)&(month.iloc[x,9] > 0)&(month.iloc[x,10] > 0)&(month.iloc[x,11] > 0)&(month.iloc[x,12] > 0)&(month.iloc[x,13] > 0):
        val = 1 # ACTIVE USER    
    elif (month.iloc[x,1] > 0)&(month.iloc[x,2] > 0)&(month.iloc[x,3] > 0)&(month.iloc[x,4] > 0)&(month.iloc[x,5] > 0)&(month.iloc[x,6] > 0)&(month.iloc[x,7] > 0)&(month.iloc[x,8] > 0)&(month.iloc[x,9] > 0)&(month.iloc[x,10] > 0)&(month.iloc[x,11] > 0)&(month.iloc[x,12] > 0)&(month.iloc[x,13] == 0):
        val = 0  # JUST CHURNED   
    elif (month.iloc[x,1] > 0)&(month.iloc[x,2] > 0)&(month.iloc[x,3] > 0)&(month.iloc[x,4] > 0)&(month.iloc[x,5] > 0)&(month.iloc[x,6] > 0)&(month.iloc[x,7] > 0)&(month.iloc[x,8] > 0)&(month.iloc[x,9] > 0)&(month.iloc[x,10] > 0)&(month.iloc[x,11] > 0)&(month.iloc[x,12] == 0):
        val = 2  # ALREADY CHURNED 
    elif (month.iloc[x,12] == 0)&(month.iloc[x,11] == 0)&(month.iloc[x,10] == 0):
        val = 2  # ALREADY CHURNED
    elif (month.iloc[x,12] > 0)&(month.iloc[x,11] == 0)&(month.iloc[x,10] == 0)&(month.iloc[x,9] == 0):
        val = 3  # NEW CUSTOMER (DON'T INVOLVE INTO MODEL -DROP- )
    else:
        val = 1 # REST COULD BE CONSIDERED ACTIVE
    User_Status.append(val)
User_Status = pd.DataFrame(User_Status,columns = {"User_Status"})
# Integrate user definitions with pre generated time series dataset 
ts3_11 = pd.concat([month11,User_Status],axis=1)
print(ts3_11.groupby('User_Status')['2019-12'].count())

### CREATE A CONSOLIDATED DATAFRAME (ENTIRE 11 MONTH CHURNED USERS + THE MOST RECENT MONTH FULL DATA)

In [ ]:
namesList = ['USER_ID','1','2','3','4','5','6','7','8','9','10','11','12','13','Recency','Frequency','User_Status']
# CREATE A CONSOLIDATED DATAFRAME (ENTIRE 11 MONTH CHURNED USERS + THE MOST RECENT MONTH FULL DATA)
ts3_1_minority = ts3_1[ts3_1.User_Status==0];ts3_1_minority.columns = namesList
ts3_2_minority = ts3_2[ts3_2.User_Status==0];ts3_2_minority.columns = namesList
ts3_3_minority = ts3_3[ts3_3.User_Status==0];ts3_3_minority.columns = namesList
ts3_4_minority = ts3_4[ts3_4.User_Status==0];ts3_4_minority.columns = namesList
ts3_5_minority = ts3_5[ts3_5.User_Status==0];ts3_5_minority.columns = namesList
ts3_6_minority = ts3_6[ts3_6.User_Status==0];ts3_6_minority.columns = namesList
ts3_7_minority = ts3_7[ts3_7.User_Status==0];ts3_7_minority.columns = namesList
ts3_8_minority = ts3_8[ts3_8.User_Status==0];ts3_8_minority.columns = namesList
ts3_9_minority = ts3_9[ts3_9.User_Status==0];ts3_9_minority.columns = namesList
ts3_10_minority = ts3_10[ts3_10.User_Status==0];ts3_10_minority.columns = namesList
ts3_11_minority = ts3_11[ts3_11.User_Status==0];ts3_11_minority.columns = namesList
ts3_11_majority = ts3_11[ts3_11.User_Status==1];ts3_11_majority.columns = namesList
consolidation_list = [ts3_1_minority,ts3_2_minority,ts3_3_minority,ts3_4_minority,ts3_5_minority,ts3_6_minority,ts3_7_minority,ts3_8_minority,ts3_9_minority,ts3_10_minority,ts3_11_minority,ts3_11_majority]
df = pd.concat(consolidation_list,axis=0)
df = df.drop_duplicates(['USER_ID'])
df.head(3)

In [ ]:
# Delete previously created dummy datasets
del ts3_1 ; del ts3_2 ; del ts3_3 ; del ts3_4 ; del ts3_5 ; del ts3_6 ; del ts3_7 ; del ts3_8 ; del ts3_9 ; del ts3_10 ;del ts3_11

In [ ]:
# Delete previously created dummy datasets
del month1;del month2;del month3;del month4;del month5;del month6;del month7;del month8;del month9;del month10;del month11 

In [ ]:
# a brief look on recency and frequency features on churn vs active users
plt.figure(figsize=(15,5))
sns.distplot(df[df.User_Status == 0].Recency,hist=False)
sns.distplot(df[df.User_Status == 1].Recency,hist=False)
plt.figure(figsize=(15,5))
sns.distplot(df[df.User_Status == 0].Frequency,hist=False)
sns.distplot(df[df.User_Status == 1].Frequency,hist=False)

### NEW FEATURE CREATION TO INCREASE ACCURACY

In [ ]:
# New Feature  - Median of last 6 months
df['L6_Month_Median'] = df.iloc[:,7:13].median(axis=1)
# New Feature  - Minimum of last 3 months
df['L3_Month_Min'] = df.iloc[:,10:13].min(axis=1)
# New Feature  - Standard Deviation of last 6 months
df['L6_Month_STD'] = df.iloc[:,7:13].std(axis=1)
df.head(3)

### TIME SERIES PREDICTION TO PREDICT EACH USER'S FUTURE RECEIPT UPLOAD WITH EXPONENTIAL SMOOTHING

In [ ]:
# Transform data by melting month columns into one column 
df_ts = df.loc[:,'USER_ID':'13']
df_ts = df_ts.melt(id_vars = 'USER_ID',var_name='Month',value_name='Receipt Uploaded')
df_ts.Month = df_ts.Month.astype(int)
df_ts.head(3)

In [ ]:
# Transform data by spreading User-ID's as seperate columns to make them ready for time series prediction with a loop for each
df_ts = df.loc[:,'USER_ID':'13']
df_ts = df_ts.melt(id_vars = 'USER_ID',var_name='Month',value_name='Receipt Uploaded')
df_ts = df_ts.pivot(index='Month',columns = 'USER_ID', values = 'Receipt Uploaded').reset_index()
df_ts.Month = pd.to_numeric(df_ts.Month)
df_ts = df_ts.set_index('Month').sort_values(by='Month')
df_ts.tail(3)

In [ ]:
                        
                            ######## EXPONENTIAL SMOOTHING PREDICTION LOOP FOR EACH USER ########
SES_12M = []
SES_3M = []

upper = len(df_ts.columns)
for x in range(0,upper):

    # Sımple Exponentıal Smoothıng Predictions for 12 - 6 - 3  Month Periods
    
    # Prediction based on past 12 months
    df_1  = df_ts.iloc[0:12,x]
    ses_12 = SimpleExpSmoothing(df_1).fit()
    ses_12_output = ses_12.forecast(1)
    SES_12M.append(ses_12_output)
    
    del df_1;del ses_12; del ses_12_output # delete dummy datasets

    
    # Prediction based on past 3 months
    df_5  = df_ts.iloc[9:12,x]
    ses_3 = SimpleExpSmoothing(df_5).fit()
    ses_3_output = ses_3.forecast(1)
    SES_3M.append(ses_3_output)
    
    del df_5;del ses_3; del ses_3_output # delete dummy datasets
    

In [ ]:
#K Convert prediction results into a dataframe format
ses_12 = pd.DataFrame(SES_12M).rename(columns={12: "Prediction_SES_12M"}).reset_index().iloc[:,1]
ses_3 = pd.DataFrame(SES_3M).rename(columns={3: "Prediction_SES_3M"}).reset_index().iloc[:,1]

# Combine each different prediction into one dataset
df_columns = pd.DataFrame(df_ts.columns)
list1 = [df_columns,ses_12,ses_3]
data = pd.concat(list1,axis=1)
data.head(3)

In [ ]:
# Combine existing datafarme with prediction results and create the ultimate prediction feature
cols = ['USER_ID',"12",'13','User_Status','L6_Month_Median','L3_Month_Min','L6_Month_STD','Recency','Frequency']
df_combined = df[cols].merge(data,left_on='USER_ID', right_on='USER_ID') 
df_combined = df_combined.rename(columns={'12': "Previous_Month",'13': "Predicted_Month"})
list1 = ['Prediction_SES_3M','Prediction_SES_12M'] 
# Generate the ultimate prediction column by averaging 3 different exponential smoothing prediction results
df_combined['Actual_Month_Prediction'] =  df_combined[list].mean(1)
df_combined.head()

### COMBINE CREATED TIME SERIES DATASET WITH COMPANY X DEMOGRAPHICS DATASET

In [ ]:
#K "Birth Year ID" to "Age" Conversion in order to utilize this feature in the model
demographics = pd.read_csv('demographics.csv')
def age_calculator(x):
    return (2020 - x['BIRTH_YEAR_ID'])
demographics['Age'] = demographics.apply(age_calculator,axis=1)
demographics.head(2)

#K Merge Demographics and Time-Series Datasets
drop_cols = ['BIRTH_YEAR_ID','FIRST_TRANSACTION_DT','FIRST_STATIC_START_DATE','LAST_STATIC_END_DATE']
holds = ['Actual_Month_Prediction','USER_ID','Previous_Month','User_Status','L6_Month_STD','L6_Month_Median','Recency','Frequency']
final_df = demographics.merge(df_combined[holds],how='right',left_on='USER_ID', right_on='USER_ID').drop(drop_cols,axis=1)
final_df.head(3)

### CREATE NEW FEATURES (PERIODICITY & DISCIPLINE)

In [ ]:
# Convert User ID and Basket ID into string format
purchases['USER_ID']=purchases['USER_ID'].apply(str)
purchases['BASKET_ID']=purchases['BASKET_ID'].apply(str)
# Calculate unique transactions for each user
df_periodicity=pd.DataFrame(purchases.groupby(['TRANSACTION_DT','USER_ID'])['BASKET_ID'].nunique().reset_index())
# Create dummy features to be able to calculate Periodicity and Discipline Features (Previous Transaction & Days between transactions) 
df_periodicity['PREVIOUS_TRANS']=df_periodicity.groupby(['USER_ID'])['TRANSACTION_DT'].shift()
df_periodicity['DAYS_BW_TRANS']=df_periodicity['TRANSACTION_DT']-df_periodicity['PREVIOUS_TRANS']
df_periodicity['DAYS_BW_TRANS']=df_periodicity['DAYS_BW_TRANS'].apply(lambda x: x.days)

# Use created features to distill a new dataframe with each User in seperate columns
list1 = ['TRANSACTION_DT','USER_ID','DAYS_BW_TRANS']
df_prd =  df_periodicity[list1].dropna()
df_prd = df_prd.pivot(index='TRANSACTION_DT',columns='USER_ID',values='DAYS_BW_TRANS').reset_index().sort_values(by='TRANSACTION_DT',ascending=False)

# Create a for loop for each user to calculate periodicity and discipline features
periodicity = []
discipline = []
upper = len(df_prd.columns)
for x in range(1,upper):
    
    df_1  = df_prd.iloc[:,x]
    result = df_1.dropna().iloc[0:50].mean() # last 50 transaction
    periodicity.append(result)
    dis = df_1.dropna().iloc[0:50].std() # last 50 transaction
    discipline.append(dis)

# Create a new dataset with calculated features
periodicity = pd.DataFrame(periodicity,columns=['periodicity'])
discipline = pd.DataFrame(discipline,columns=['discipline'])
users  = pd.DataFrame(df_prd.drop('TRANSACTION_DT',axis=1).columns.tolist(),columns=['USER_ID'])
list1 = [users,periodicity,discipline]
new_features = pd.concat(list1,axis=1)

# Integrate new features into existing final dataset
final_df['USER_ID']=final_df['USER_ID'].apply(str)
final_df_v2 = final_df.merge(new_features,left_on='USER_ID', right_on='USER_ID')
final_df_v2.discipline = final_df_v2.discipline * -1
final_df_v2.head(3)

In [ ]:
# delete dummy datasets
del df_periodicity;del df_prd; del final_df;del periodicity;del discipline

### CREATE NEW FEATURES (RISKY CUSTOMERS & DUMMY USER STATUS)

In [ ]:
# Create a 3D graph to explore features impact on User's activeness or churn potential
fig = px.scatter_3d(final_df_v2, x='Actual_Month_Prediction', y='periodicity', z='discipline',
              color='User_Status')
fig.show()

In [ ]:
#Create new Feature "RISKY CUSTOMERS" based on explorations above
def risky(x):
    if (x['Actual_Month_Prediction'] < 20) & (x['periodicity'] >3) & (x['discipline'] < -10):
        val=1
    else:
        val=0
    return val
final_df_v2['Risky_Customers'] = final_df_v2.apply(risky,axis=1)

#Create new Feature (Prev_Pred_Change) Difference between Predicted month and Previous Month to show trend
final_df_v2['Prev_Median_Change'] =  final_df_v2.Previous_Month - final_df_v2.L6_Month_Median

print(final_df_v2.Risky_Customers.value_counts())

In [ ]:
# Scatterplotting of several features to observe their impact on User's activeness or churn potential
sns.scatterplot('Actual_Month_Prediction','Recency',hue='User_Status',data=final_df_v2)

In [ ]:
#Create new Feature "dummy_user_status" based on explorations above
def dummy_user_status(x):
    if (x['Actual_Month_Prediction'] < 0):
        val='Churn'
    elif (x['Recency'] > 25):
        val='Churn'
    elif (x['discipline'] < -20):
        val='Churn'
    elif (x['Actual_Month_Prediction'] > 15) & (x['Recency'] > 20):
        val='Churn'
    elif (x['Actual_Month_Prediction'] > 10) & (x['Recency'] > 25):
        val='Churn'
    elif (x['Actual_Month_Prediction'] > 30) & (x['Recency'] < 4):
        val='Active'
    else:
        val='Unclear'
    return val
final_df_v2['dummy_user_status'] = final_df_v2.apply(dummy_user_status,axis=1)
final_df_v2.groupby('dummy_user_status').count()

In [ ]:
#New Feature -  User Channel Based Transaction Percentages 
# Create a new feature indicating user's channel preference
list1 = ['USER_ID','CHANNEL_ID'];channel_trend = purchases[list1];channel_trend['N'] = 1
channel_trend = pd.DataFrame(pd.pivot_table(channel_trend, values = 'N', index=['USER_ID'], 
                columns = ['CHANNEL_ID'],aggfunc='count')).fillna(0)
channel_trend['Total_Channel_ Count'] = channel_trend.sum(1)
channel_trend = channel_trend.div( channel_trend.iloc[:,-1], axis=0).drop('Total_Channel_ Count',axis=1).reset_index()
channel_trend['USER_ID']=channel_trend['USER_ID'].apply(str)
list1= ['USER_ID','User_Status']

# Since there are tons of channels for the sake of simplicity elimination implemented based on their correlation to dependent variable
channel_trend = final_df_v2[list1].merge(channel_trend,left_on='USER_ID', right_on='USER_ID') 
corr = pd.DataFrame(channel_trend.corr().User_Status).sort_values(by='User_Status')
channel_features = corr[(corr.User_Status > 0.03)|(corr.User_Status < -0.03)].reset_index()['index'].tolist()
channel_features.remove('User_Status');channel_features.append('USER_ID')
final_df_v3 = final_df_v2.merge(channel_trend[channel_features],left_on='USER_ID', right_on='USER_ID') 
final_df_v3.head(3)

In [ ]:
#New Feature -  User Retail ID Based Transaction Percentages 
# Create a new feature indicating user's retailer preference
list1 = ['USER_ID','RETAILER_ID'];retail_trend = purchases[list1];retail_trend['N'] = 1
retail_trend = pd.DataFrame(pd.pivot_table(retail_trend, values = 'N', index=['USER_ID'], 
                columns = ['RETAILER_ID'],aggfunc='count')).fillna(0)

#Eliminating small scale retailers and manipulating this dataframe to be compatible to a merge with main dataframe
retail_filter = pd.DataFrame(retail_trend.sum(axis=0)).reset_index()
retail_filter = retail_filter[retail_filter[0] > 50000]["RETAILER_ID"].tolist()
retail_filter.append('USER_ID')
retail_trend['Total_Retail_Count'] = retail_trend.sum(1)
retail_trend = retail_trend.div( retail_trend.iloc[:,-1], axis=0).drop('Total_Retail_Count',axis=1).reset_index()
retail_trend['USER_ID']=retail_trend['USER_ID'].apply(str)

# Integrate recently generated feature to main dataframe
list1= ['USER_ID','User_Status']
retail_trend = final_df_v3[list1].merge(retail_trend[retail_filter],left_on='USER_ID', right_on='USER_ID') 
corr = pd.DataFrame(retail_trend.corr().User_Status).sort_values(by='User_Status')
retail_features = corr[(corr.User_Status > 0.03)|(corr.User_Status < -0.03)].reset_index()['index'].tolist()
retail_features.append('USER_ID') ; retail_features.remove('User_Status')
final_df_v3 = final_df_v3.merge(retail_trend[retail_features],left_on='USER_ID', right_on='USER_ID') 
final_df_v3.head(3)

#### OUTLIER REMOVAL

In [ ]:
# Create a scatterplot to explore major features' and their impact on the dependent variable
plt.figure(figsize=(18,7))
sns.stripplot(x='Recency',y='Actual_Month_Prediction',hue='Risky_Customers',data=final_df_v3[final_df_v3.User_Status == 0])

In [ ]:
# Outlier removal conducted with trial & error based on the highest model performance
Churn_Data = final_df_v3[final_df_v3.User_Status == 0]
Churn_Data_Outliers = Churn_Data[(Churn_Data.Previous_Month > 25) & (Churn_Data.Recency > 13)] 

In [ ]:
# Check out the shapes of before/after datasets to see how many rows are eliminated
print(final_df_v3.shape)
list = [final_df_v3,Churn_Data_Outliers]
final_df_v4 = pd.concat(list).drop_duplicates(keep=False)
print(final_df_v4.shape)

In [ ]:
# Age (No Major difference between actives and churns, so I am not going to segment this feature)
sns.distplot(final_df_v4[final_df_v4['User_Status'] == 0].Age,hist=False)
sns.distplot(final_df_v4[final_df_v4['User_Status'] == 1].Age,hist=False)

# PREDICTION WITH FINALIZED DATASET 

In [ ]:
#K Generate Dummy Variables for Categorical Features
dummies = ['dummy_user_status','STATE_ID','RURAL_CODE','CENSUS_DIVISION_NAME','EMPLOYMENT_ID','GENDER_ID','EDUCATION_ID','ETHNICITY_ID','HH_SIZE_ID','INCOME_NEW_ID','RURAL_CODE']
final_df_v5 = pd.DataFrame(pd.get_dummies(final_df_v4,columns=dummies,drop_first=True))
final_df_v5.head(5)

In [ ]:
# Writing Model Data 
final_df_v5.to_csv('Model_Data.csv',index=False)

In [ ]:
# Start making predictions with prominent ML models
##### RANDOM FOREST #######
drop = ['User_Status']
X = final_df_v5.drop(drop,axis=1)
y = final_df_v5['User_Status']
y = np.array(y)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=25)

# 200 trees
rfc = RandomForestClassifier(n_estimators=200)
rfc.fit(X_train,y_train)
X_train.shape

# Prediction on Test Dataset
rf_pred = rfc.predict(X_test)
from sklearn.metrics import classification_report,confusion_matrix
print(classification_report(y_test,rf_pred))

#### REGULARIZATION TO ELIMINATE REDUNDANT VARIABLES

In [ ]:
# Regularization for Feature Importance
drops = ['User_Status']
X = final_df_v5.drop(drops,axis=1)
y = final_df_v5['User_Status']
y = np.array(y)
# Used l1 penalty with 0.005 (trial & error)
lsvc = LinearSVC(C=0.005, penalty="l1", dual=False).fit(X, y) 
regularized_model = SelectFromModel(lsvc, prefit=True)
X_new = regularized_model.transform(X)
X_regularized = pd.DataFrame(X_new)
Regularized_Features = X.columns[(regularized_model.get_support())]
X_regularized = X[Regularized_Features]
X_regularized.shape

In [ ]:
# Re-predict with the eliminated features
##### RANDOM FOREST #######
X = final_df_v5[Regularized_Features]
y = final_df_v5['User_Status']
y = np.array(y)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=25)

# 200 trees
rfc = RandomForestClassifier(n_estimators=200,class_weight=None)
rfc.fit(X_train,y_train)
X_train.shape
# Prediction on Test Dataset
rf_pred = rfc.predict(X_test)
from sklearn.metrics import classification_report,confusion_matrix
print(classification_report(y_test,rf_pred))

#### CLUSTERING

In [ ]:
# Define Top Features to process clustering
# Used random forest classifier package to see best performing features
from sklearn.ensemble import RandomForestClassifier
df = final_df_v5[Regularized_Features]
feat_importances = pd.Series(rfc.feature_importances_, index=df.columns)
feat_importances.nlargest(10).plot(kind='barh')

In [ ]:
# Create 'Cluster' Feature based on top performing features calculated above
top_features = ['discipline','Recency','Previous_Month','periodicity','Actual_Month_Prediction']
df_cluster_t = final_df_v5[top_features]
mms_t = MinMaxScaler()
mms_t.fit(df_cluster_t)
data_transformed_t = mms_t.transform(df_cluster_t)

# Processing k-means clustering for up to 10 clusters
Sum_of_squared_distances = []
K = range(1,10)
for k in K:
    km = KMeans(n_clusters=k)
    km = km.fit(data_transformed_t)
    Sum_of_squared_distances.append(km.inertia_)
 
# Plot the result to identify optimal K
plt.plot(K, Sum_of_squared_distances, 'bx-')
plt.xlabel('k')
plt.ylabel('Sum_of_squared_distances')
plt.title('Elbow Method For Optimal k')
plt.show()

In [ ]:
# 2 or 3 clusters seems bets,I choose to go ahead with 3 clusters
kmeans = KMeans(n_clusters=3)
kmeans.fit(data_transformed_t)
Clusters = pd.DataFrame(kmeans.labels_)
Clusters.columns = ['Clusters']

# Integrate cluster columns to main model dataframe
Regularized_Features_2 = Regularized_Features.tolist() ; Regularized_Features_2.append('USER_ID'); Regularized_Features_2.append('User_Status')
Regularized_Features_2.remove('RURAL_CODE_Urban')
final_df_v6 = pd.concat([final_df_v5[Regularized_Features_2],Clusters],axis=1,join='inner')
dummies = ['Clusters']
final_df_v6 = pd.DataFrame(pd.get_dummies(final_df_v6,columns=dummies,drop_first=True))
final_df_v6.tail(2)

#### PREDICT WITH SELECTED FEATURES AND CLUSTERING

In [ ]:
# Re-predict with eliminated features and added clusters
##### RANDOM FOREST #######
drops = ['USER_ID','User_Status'] 
X = final_df_v6[Regularized_Features_2].drop(drops,axis=1)
y = final_df_v6['User_Status']
y = np.array(y)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=25)

# 200 trees
rfc = RandomForestClassifier(n_estimators=200,class_weight=None)
rfc.fit(X_train,y_train)
X_train.shape
# Prediction on Test Dataset
rf_pred = rfc.predict(X_test)
from sklearn.metrics import classification_report,confusion_matrix
print(classification_report(y_test,rf_pred))

In [ ]:
# Predict with other ML algorithms
### XGBOOST ###
# Paramater tuning based trial and error results
model1 = xgb.XGBClassifier(max_depth=9,booster='gbtree',learning_rate=0.065)
X_train = np.array(X_train);y_train = np.array(y_train);X_test = np.array(X_test);y_test = np.array(y_test)
train_model1 = model1.fit(X_train,y_train)
xgb_pred = train_model1.predict(X_test)
print(classification_report(y_test,xgb_pred))

### DOWN SAMPLING MAJORITY 

In [ ]:
# Since the active user class is predominant in the dataset, downsampling applied in order to balance these two classes

####### RANDOM FOREST REGULARIZED & CLUSTERED & BALANCED ######
drops = ['USER_ID','User_Status'] 
X = final_df_v6[Regularized_Features_2].drop(drops,axis=1)
y = final_df_v6['User_Status']
y = np.array(y)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=25)

# optimal ratio is found by trials
from imblearn.under_sampling import RandomUnderSampler
ratio = {0:4402,1:12000}
rus = RandomUnderSampler(sampling_strategy=ratio)
X_train = np.array(X_train)
X_train, y_train = rus.fit_sample(X_train, y_train)

# 200 trees
rfc = RandomForestClassifier(n_estimators=50,class_weight=None)
rfc.fit(X_train,y_train)
print(X_train.shape)

# Prediction on Test Dataset
rf_pred = rfc.predict(X_test)
from sklearn.metrics import classification_report,confusion_matrix
print(classification_report(y_test,rf_pred))

In [ ]:
# This is for a later calculation for ShinyApp at the bottom

y_test_df = pd.DataFrame(y_test).rename(columns={0: "User_Status"})
X_test_df = X_test.reset_index()
test_df = pd.concat([X_test_df,y_test_df],axis=1).drop('index',axis=1)
test_df.tail(5)

In [ ]:
# Predict with other ML algorithms
### XGBOOST ###
model1 = xgb.XGBClassifier(max_depth=9,booster='gbtree',learning_rate=0.065)
X_train = np.array(X_train);y_train = np.array(y_train);X_test = np.array(X_test);y_test = np.array(y_test)
train_model1 = model1.fit(X_train,y_train)
xgb_pred = train_model1.predict(X_test)
print(classification_report(y_test,xgb_pred))

In [ ]:
# Predict with other ML algorithms
### lOGISTIC REGRESSION ###
log = LogisticRegression(solver='liblinear')
log.fit(X_train,y_train)

# Prediction on Test Dataset
log_pred = log.predict(X_test)
print(classification_report(y_test,log_pred))

In [ ]:
# Mixed Model of Random Forest and XGBoost
mixed_pred = (rf_pred +xgb_pred) / 2
mixed_pred = mixed_pred.astype('uint8')
print(classification_report(y_test,mixed_pred))

In [ ]:
#Neural Network Model 

X_train_nn = np.array(X_train);y_train_nn = np.array(y_train);X_test_nn = np.array(X_test);y_test_nn = np.array(y_test)
from sklearn.utils import class_weight
class_weights = class_weight.compute_class_weight('balanced',np.unique(y_train), y_train)                                                                           
scaler = MinMaxScaler();X_train_nn = scaler.fit_transform(X_train);X_test_nn = scaler.transform(X_test)

# Patience is set to 25 and dropout is determined as 0.1
early_stop = EarlyStopping(monitor = 'val_loss', mode = 'min', verbose = 1, patience = 25)
model = Sequential()
model.add(Dense(X_train.shape[1], activation='relu', input_dim=X_train.shape[1]))
model.add(Dropout(0.1))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer = 'adam', loss='binary_crossentropy',metrics=['accuracy'],learning_rate=0.0001)
model.fit(X_train,y_train,epochs = 1000, validation_data = (X_test_nn, y_test_nn), callbacks=[early_stop],verbose = 0,class_weight=None)  

In [ ]:
# Prediction on Test Dataset
# Cutoff rate 0.52 yields the optimal results for precision and recall
DL_pred = model.predict(X_test)
nn_pred = (DL_pred > 0.52)
print(classification_report(y_test,nn_pred))

In [ ]:
from sklearn.metrics import f1_score
# Calculated exact F1 Score
f1_score(y_test, nn_pred, average=None)[0]

In [ ]:
# Converted prediction results to uint format
nn_pred = np.reshape(nn_pred,(7943,)).astype('uint8')

In [ ]:
### Mixed Model 2 (XGBoost and Neural Network Models)
mixed = (xgb_pred +nn_pred) / 2
mixed = mixed.astype('uint8')
print(classification_report(y_test,mixed))

### HEREAFTER PART IS FOR SHINYAPP

In [ ]:
prediction = pd.DataFrame(DL_pred).rename(columns={0: "Model Prediction"})
list_2 = [test_df,prediction]
assessment = pd.concat(list_2,axis=1)

In [ ]:
sns.distplot(assessment[assessment.User_Status == 0]['Model Prediction'],hist=False)
sns.distplot(assessment[assessment.User_Status == 1]['Model Prediction'],hist=False)

In [ ]:
labels = [1,2,3,4,5]
assessment['binned'] =  pd.cut(assessment['Model Prediction'], bins=5, labels=labels)
assessment.head(3)

In [ ]:
df_churn = pd.pivot_table(assessment, values='Model Prediction', index=['binned'],columns=['User_Status'], aggfunc='count').reset_index()
df_churn['Churn Risk'] =df_churn[0]/(df_churn[0] + df_churn[1]) ; df_churn['Churn Risk'] =(df_churn['Churn Risk'].round(2) * 100).round(0)
df_churn

In [ ]:
def description(x):
    if x['binned'] == 1:
        val = 'High Churn Risk ~95%'
    elif x['binned'] == 2:
        val = 'High-Moderate Churn Risk ~60%'
    elif x['binned'] == 3:
        val = 'Moderate Churn Risk ~40%'
    elif x['binned'] == 4:
        val = 'Low-Moderate Churn Risk ~15%'
    else:
        val = 'Low Churn Risk ~2%'
    return val
assessment['Description'] = assessment.apply(description,axis=1)
assessment.head(3)

In [ ]:
list_1 = ['Recency','Frequency','periodicity','discipline','binned','Description','Model Prediction']
assessment[list_1].to_csv('Assessment.csv',index=False)